<h1 align='Center'>Segmenting and Clustering Neighborhoods in the city of Toronto Canada</h1>


### Scraping Postal Codes from Wikipidea

In [1]:
#loading all the necessary libraries
from bs4 import BeautifulSoup
import requests
import urllib # for working with URLs
import pandas as pd

In [2]:
# getting the html content of the wikipedia content
source =urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(source, 'lxml')
div_table = soup.find('div', class_='mw-parser-output')
# getting the from the html script
table = div_table.find('table', class_='wikitable sortable')
# getting of the rows of the table
ligne = table.find_all('tr')

<h4> This function generate a dataframe of postal code </h4>
<h7> The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</h7>
<h7> Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</h7>
<h7> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</h7>
<h7> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</h7>

In [8]:
def createPostalDataFrame():
    liste_postal_code = [] # this list will serve for creation the dataframe
    for element in ligne:
        to_string_element = str(element)
        if '<td>' in to_string_element:
            contenu = to_string_element.split('<td>')
            if not 'Not assigned' in str(contenu[2])[:- 6]: # Ignore cells with a borough that is Not assigned
                PostalCode = str(contenu[1])[:- 6]
                if '<a' in str(contenu[2])[:- 6]:
                    btfsoup_Borough = BeautifulSoup(str(contenu[2])[:- 6])
                    Borough = btfsoup_Borough.find('a')['title']
                else:
                    Borough = str(contenu[2])[:- 6]
                
                if 'Not assigned' in str(contenu[3])[:- 10]: # If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
                    Neighborhood = Borough
                else:
                    if '<a' in str(contenu[3])[:- 10]:
                        btfsoup_Neighborhood = BeautifulSoup((contenu[3])[:- 10])
                        Neighborhood = btfsoup_Neighborhood.find('a')['title']
                    else:
                        Neighborhood = str(contenu[3])[:-12]
                    
                liste_postal_code.append([PostalCode, Borough, Neighborhood])
                
    return liste_postal_code

In [7]:
dataPostal = pd.DataFrame(createPostalDataFrame(), columns=['PostalCode', 'Borough', 'Neighborhood'])
dataPostal.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront (Toronto)
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Group the table by PostalCode/Borough, Neighbourhood belonging to same borough will be combined in 'Neighbourhood' column as separated with 'comma'

In [9]:
dataPostalClean = dataPostal.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
dataPostalClean.head()

,PostalCode,Borough,Neighborhood
0,M1B,"Scarborough, Toronto","Rouge, Toronto, Malvern, Toronto"
1,M1C,"Scarborough, Toronto","Highland Creek (Toronto), Rouge Hill, Port Uni..."
2,M1E,"Scarborough, Toronto","Guildwoo, Morningside, Toronto, West Hill, Tor..."
3,M1G,"Scarborough, Toronto","Woburn, Toronto"
4,M1H,"Scarborough, Toronto",Cedarbra


In [14]:
dataPostalClean.to_csv('Toronto_Postal_data.csv', index=False)

In [15]:
dataPostalClean.shape

(103, 3)